<a href="https://colab.research.google.com/github/summerolmstead/Federal-Reserve-CPI/blob/main/Federal_Reserve_CPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Summer Olmstead

Part 2 of Final Project Data Visualization

Macroeconomic Finance Theme

In [1]:
!pip install pandas_datareader

import pandas_datareader.data as web
import pandas as pd
import plotly.express as px


In [2]:
#date range same as before
start = '2010-01-01'
end = '2025-04-01'

#CPI data from FRED using our datatreader to not  have to download
cpi = web.DataReader('CPIAUCNS', 'fred', start, end) #'CPIAUCNS' = the CPI All Urban Consumers, Not Seasonally Adjusted series code and  fred= tells pandas_datareader you’re using the FRED source

#must reset index so 'DATE' becomes a column that we can easily code with
cpi = cpi.reset_index()

#rename for clarity
cpi = cpi.rename(columns={'DATE': 'Date', 'CPIAUCNS': 'CPI'})

In [3]:
#stats
cpi.describe()

,Date,CPI
count,183,183.000000
mean,2017-07-31 23:44:15.737704960,255.195328
min,2010-01-01 00:00:00,216.687000
25%,2013-10-16 12:00:00,233.651500
50%,2017-08-01 00:00:00,245.519000
75%,2021-05-16 12:00:00,270.445500
max,2025-03-01 00:00:00,319.799000
std,NaN,29.405097


In [4]:
#shape to get dimensions
cpi.shape

(183, 2)

In [5]:
# YoY - year over year percent change to see how much inflation grew from the previous year
cpi['YoY_Change'] = cpi['CPI'].pct_change(periods=12) * 100

In [6]:
#now lets plot using plotly the data with the range slider very basic first
fig = px.line(
    cpi,
    x='Date',
    y='CPI',
    title='Consumer Price Index (CPI) Over Time',
    labels={'CPI': 'CPI Value (All Urban Consumers)', 'Date': 'Date'}
)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [7]:
#now out of curiousity lets do the yoy to see the shape to see if one year rather than others has a higher inflation shock
fig = px.line(
    cpi,
    x='Date',
    y='YoY_Change',
    title='Year-over-Year CPI Inflation Rate',
    labels={'YoY_Change': 'Percent Change YoY (%)', 'Date': 'Date'}
)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

Dual Line Chart with both YoY and CPI - common in Federal Reserve Bank dashboards (like FRED)

In [8]:
import plotly.graph_objects as go

#now lets just graph both
#initalie figure
fig = go.Figure()

#first the line for CPI value line
fig.add_trace(go.Scatter(
    x=cpi['Date'],
    y=cpi['CPI'],
    mode='lines',
    name='CPI (Index)',
    line=dict(color='blue')
))

#next we need the line for YoY % change
fig.add_trace(go.Scatter(
    x=cpi['Date'],
    y=cpi['YoY_Change'],
    mode='lines',
    name='YoY Inflation Rate (%)',
    line=dict(color='red'),
    yaxis='y2'  #need to use a second y-axis
))

#update layout with two y-axes to have both!
fig.update_layout(
    title='CPI and Year-over-Year Inflation Rate',
    xaxis=dict(
        title='Date',
        rangeslider=dict(visible=True),
        rangeselector=dict(
            buttons=list([ #making a 5 year increament backward buttons
                dict(count=1, label="1Y", step="year", stepmode="backward"),
                dict(count=5, label="5Y", step="year", stepmode="backward"),
                dict(count=10, label="10Y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        )
    ),#defining cpi line to be blue
    yaxis=dict(
        title='CPI (Index)',
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue')
    ),
    yaxis2=dict(#defining yoy line to be red
        title='YoY Inflation (%)',
        overlaying='y',
        side='right',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    legend=dict(x=0.01, y=0.99)
)

fig.show()
fig.write_html("cpi_and_yoy.html") #downloading

This dual-line chart shows both the raw CPI index and the year-over-year percent change in inflation. The idea is to let people see not just how prices have trended over time, but also how fast they’re rising. So while the CPI line gives the actual price level (which generally increases over time), the YoY change shows how sharply or slowly that growth is happening. When the YoY line spikes, that usually means prices are rising fast across the board — groceries, rent, gas, etc. From a macro perspective, high inflation can erode purchasing power and lead to policy responses like interest rate hikes. So this chart helps connect the dots between what people feel (like higher prices) and the actual economic data.